# Data Extraction Process

In [1]:
## Loading Libraries
import pandas as pd
import numpy as np

In [2]:
## Download Data from: https://www.kaggle.com/datasets/paololol/league-of-legends-ranked-matches

## Loading Required Datasets
matches = pd.read_csv("C:\\Users\\Admin\\Desktop\\upwork_jobs\\lol_project\\data\\matches.csv")
champs = pd.read_csv("C:\\Users\\Admin\\Desktop\\upwork_jobs\\lol_project\\data\\champs.csv")
participants = pd.read_csv("C:\\Users\\Admin\\Desktop\\upwork_jobs\\lol_project\\data\\participants.csv")
stats1 = pd.read_csv("C:\\Users\\Admin\\Desktop\\upwork_jobs\\lol_project\\data\\stats1.csv")
stats2 = pd.read_csv("C:\\Users\\Admin\\Desktop\\upwork_jobs\\lol_project\\data\\stats2.csv")

c:\Users\Admin\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (52) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Quick Look at Datasets

In [4]:
matches.head()

,id,gameid,platformid,queueid,seasonid,duration,creation,version
0,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675
1,11,3187425281,EUW1,420,8,1693,1495066760778,7.10.187.9675
2,12,3187269801,EUW1,420,8,1482,1495053375889,7.10.187.9675
3,13,3187252065,EUW1,420,8,1954,1495050993613,7.10.187.9675
4,14,3187201038,EUW1,420,8,2067,1495047893400,7.10.187.9675


In [5]:
champs.head()

,name,id
0,Jax,24
1,Sona,37
2,Tristana,18
3,Varus,110
4,Fiora,114


In [6]:
participants.head()

,id,matchid,player,championid,ss1,ss2,role,position
0,9,10,1,19,4,11,NONE,JUNGLE
1,10,10,2,267,3,4,DUO_SUPPORT,BOT
2,11,10,3,119,7,4,DUO_CARRY,BOT
3,12,10,4,114,12,4,SOLO,TOP
4,13,10,5,112,4,3,SOLO,MID


In [7]:
stats1.head()

,id,win,item1,item2,item3,item4,item5,item6,trinket,kills,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,9,0,3748,2003,3111,3053,1419,1042,3340,6,...,69,42,27,610,13,0,0,10,0,0
1,10,0,2301,3111,3190,3107,0,0,3364,0,...,1,1,0,211,14,1,0,17,3,0
2,11,0,1055,3072,3006,3031,3046,1036,3340,7,...,3,1,2,182,14,1,0,13,5,0
3,12,0,1029,3078,3156,1001,3053,0,3340,5,...,6,6,0,106,15,0,0,3,0,0
4,13,0,3020,1058,3198,3102,1052,1026,3340,2,...,4,3,1,159,15,0,0,10,0,0


In [4]:
## Combine both stat tables
total_stats = stats1.append(stats2)

In [9]:
total_stats.head()

,id,win,item1,item2,item3,item4,item5,item6,trinket,kills,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,9,0,3748,2003,3111,3053,1419,1042,3340,6,...,69,42,27,610,13,0,0,10,0,0
1,10,0,2301,3111,3190,3107,0,0,3364,0,...,1,1,0,211,14,1,0,17,3,0
2,11,0,1055,3072,3006,3031,3046,1036,3340,7,...,3,1,2,182,14,1,0,13,5,0
3,12,0,1029,3078,3156,1001,3053,0,3340,5,...,6,6,0,106,15,0,0,3,0,0
4,13,0,3020,1058,3198,3102,1052,1026,3340,2,...,4,3,1,159,15,0,0,10,0,0


## Data Cleaning

In [5]:
## Joing Participants and their stats
player_df = pd.merge(
    participants,
    total_stats,
    how='left',
    on = ['id'],
    suffixes=('','_y'))

In [6]:
## Joining Participants, their stats and the champ they used
player_df = pd.merge(
    player_df,
    champs,
    how='left',
    left_on = 'championid',
    right_on = 'id',
    suffixes=('','_y'))

In [7]:
## Joining Participants, their stats, the champ they used, and information about the match
player_df = pd.merge(
    player_df,
    matches,
    how='left',
    left_on = 'matchid',
    right_on = 'id',
    suffixes=('','_y'))

In [8]:
## Generate a universal role:
# If there's a bot, give the role the bot played (DUO Support or Duo Carry) 
def final_position(col):
    if col['role'] in ('DUO_SUPPORT', 'DUO_CARRY'):
        return col['role']
    else:
        return col['position']

In [9]:
player_df['adjposition'] = player_df.apply(final_position, axis = 1)

In [10]:
## Preview results
player_df.head()

,id,matchid,player,championid,ss1,ss2,role,position,win,item1,...,id_y,id_y,gameid,platformid,queueid,seasonid,duration,creation,version,adjposition
0,9,10,1,19,4,11,NONE,JUNGLE,0.0,3748.0,...,19,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,JUNGLE
1,10,10,2,267,3,4,DUO_SUPPORT,BOT,0.0,2301.0,...,267,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_SUPPORT
2,11,10,3,119,7,4,DUO_CARRY,BOT,0.0,1055.0,...,119,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_CARRY
3,12,10,4,114,12,4,SOLO,TOP,0.0,1029.0,...,114,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,TOP
4,13,10,5,112,4,3,SOLO,MID,0.0,3020.0,...,112,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,MID


In [11]:
## Create labels to properly identify team
player_df['team'] = player_df['player'].apply(lambda x: 'blue' if x <= 5 else 'red')

## Get team + the role the player played.
player_df['team_role'] = player_df['team'] + ' - ' + player_df['adjposition']

In [12]:
player_df.head(10)

,id,matchid,player,championid,ss1,ss2,role,position,win,item1,...,gameid,platformid,queueid,seasonid,duration,creation,version,adjposition,team,team_role
0,9,10,1,19,4,11,NONE,JUNGLE,0.0,3748.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,JUNGLE,blue,blue - JUNGLE
1,10,10,2,267,3,4,DUO_SUPPORT,BOT,0.0,2301.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_SUPPORT,blue,blue - DUO_SUPPORT
2,11,10,3,119,7,4,DUO_CARRY,BOT,0.0,1055.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_CARRY,blue,blue - DUO_CARRY
3,12,10,4,114,12,4,SOLO,TOP,0.0,1029.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,TOP,blue,blue - TOP
4,13,10,5,112,4,3,SOLO,MID,0.0,3020.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,MID,blue,blue - MID
5,14,10,6,72,11,4,NONE,JUNGLE,1.0,1400.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,JUNGLE,red,red - JUNGLE
6,15,10,7,3,4,12,SOLO,TOP,1.0,3025.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,TOP,red,red - TOP
7,16,10,8,103,14,4,SOLO,MID,1.0,3135.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,MID,red,red - MID
8,17,10,9,222,7,4,DUO_CARRY,BOT,1.0,3046.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_CARRY,red,red - DUO_CARRY
9,18,10,10,161,14,4,DUO_SUPPORT,BOT,1.0,1058.0,...,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675,DUO_SUPPORT,red,red - DUO_SUPPORT


Removing duplicates

In [20]:
positions = player_df['team_role'].unique()[0:10]

In [22]:
## Remove records where we have duplicated team role in a match:
# - Go through each team role
# - look for records where they are duplicated
# - keep of the duplicated records
remove_index = []
for position in list(positions):
    to_remove = player_df[player_df['team_role'] == position].groupby('matchid').agg({'team_role':'count'})
    remove_index.extend(to_remove[to_remove['team_role'] != 1].index.values)

In [27]:
## Remove unclassified BOT, as correct ones should be DUO_SUPPORT OR DUO_CARRY
remove_index.extend(player_df[player_df['adjposition'] == 'BOT']['matchid'].unique())
remove_index = list(set(remove_index))

In [28]:
print('# matches in dataset before cleaning:{}'.format(player_df['matchid'].nunique()))
player_df = player_df[~player_df['matchid'].isin(remove_index)]
print('# matches in dataset after cleaning: {}'.format(player_df['matchid'].nunique()))

# matches in dataset before cleaning:184069
# matches in dataset after cleaning: 148638


In [29]:
player_df.columns

Index(['id', 'matchid', 'player', 'championid', 'ss1', 'ss2', 'role',
       'position', 'win', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
       'trinket', 'kills', 'deaths', 'assists', 'largestkillingspree',
       'largestmultikill', 'killingsprees', 'longesttimespentliving',
       'doublekills', 'triplekills', 'quadrakills', 'pentakills',
       'legendarykills', 'totdmgdealt', 'magicdmgdealt', 'physicaldmgdealt',
       'truedmgdealt', 'largestcrit', 'totdmgtochamp', 'magicdmgtochamp',
       'physdmgtochamp', 'truedmgtochamp', 'totheal', 'totunitshealed',
       'dmgselfmit', 'dmgtoobj', 'dmgtoturrets', 'visionscore', 'timecc',
       'totdmgtaken', 'magicdmgtaken', 'physdmgtaken', 'truedmgtaken',
       'goldearned', 'goldspent', 'turretkills', 'inhibkills',
       'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
       'enemyjunglekills', 'totcctimedealt', 'champlvl', 'pinksbought',
       'wardsbought', 'wardsplaced', 'wardskilled', 'firstblood', 'name

In [30]:
## Select Columns will be working with
required_columns = [
    'id', 'matchid', 'player', 'name','platformid', 'adjposition',
    'team_role', 'creation', 'version','seasonid','duration',
    'ss1', 'ss2', 'role','position', 'win',
    'item1', 'item2','item3', 'item4', 'item5', 'item6','trinket', 
    'kills', 'deaths', 'assists', 'largestkillingspree',
    'largestmultikill', 'killingsprees', 'longesttimespentliving',
    'doublekills', 'triplekills', 'quadrakills', 'pentakills',
    'legendarykills','totdmgdealt', 'magicdmgdealt', 'physicaldmgdealt',
    'truedmgdealt', 'largestcrit', 'totdmgtochamp', 'magicdmgtochamp',
    'physdmgtochamp', 'truedmgtochamp', 'totheal', 'totunitshealed',
    'dmgselfmit', 'dmgtoobj', 'dmgtoturrets', 'visionscore', 'timecc',
    'totdmgtaken', 'magicdmgtaken', 'physdmgtaken', 'truedmgtaken',
    'goldearned','goldspent', 'turretkills', 'inhibkills',
    'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
    'enemyjunglekills', 'totcctimedealt', 'champlvl', 'pinksbought',
    'wardsbought','wardsplaced', 'wardskilled', 'firstblood'
    ]

In [31]:
lol_ranked_df = player_df[required_columns]

In [32]:
## Preview final data
lol_ranked_df.head(10)

,id,matchid,player,name,platformid,adjposition,team_role,creation,version,seasonid,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,9,10,1,Warwick,EUW1,JUNGLE,blue - JUNGLE,1495068946860,7.10.187.9675,8,...,69.0,42.0,27.0,610.0,13.0,0.0,0,10.0,0.0,0.0
1,10,10,2,Nami,EUW1,DUO_SUPPORT,blue - DUO_SUPPORT,1495068946860,7.10.187.9675,8,...,1.0,1.0,0.0,211.0,14.0,1.0,0,17.0,3.0,0.0
2,11,10,3,Draven,EUW1,DUO_CARRY,blue - DUO_CARRY,1495068946860,7.10.187.9675,8,...,3.0,1.0,2.0,182.0,14.0,1.0,0,13.0,5.0,0.0
3,12,10,4,Fiora,EUW1,TOP,blue - TOP,1495068946860,7.10.187.9675,8,...,6.0,6.0,0.0,106.0,15.0,0.0,0,3.0,0.0,0.0
4,13,10,5,Viktor,EUW1,MID,blue - MID,1495068946860,7.10.187.9675,8,...,4.0,3.0,1.0,159.0,15.0,0.0,0,10.0,0.0,0.0
5,14,10,6,Skarner,EUW1,JUNGLE,red - JUNGLE,1495068946860,7.10.187.9675,8,...,111.0,81.0,30.0,808.0,16.0,0.0,0,8.0,2.0,0.0
6,15,10,7,Galio,EUW1,TOP,red - TOP,1495068946860,7.10.187.9675,8,...,6.0,0.0,6.0,179.0,16.0,1.0,0,7.0,0.0,0.0
7,16,10,8,Ahri,EUW1,MID,red - MID,1495068946860,7.10.187.9675,8,...,6.0,3.0,3.0,48.0,17.0,0.0,0,8.0,0.0,0.0
8,17,10,9,Jinx,EUW1,DUO_CARRY,red - DUO_CARRY,1495068946860,7.10.187.9675,8,...,7.0,2.0,5.0,440.0,16.0,1.0,0,6.0,1.0,1.0
9,18,10,10,VelKoz,EUW1,DUO_SUPPORT,red - DUO_SUPPORT,1495068946860,7.10.187.9675,8,...,2.0,1.0,1.0,260.0,16.0,5.0,0,25.0,7.0,0.0


In [34]:
## Remove games with incomplete data
## We just had three, so it was ok to remove them
missing_records = lol_ranked_df[lol_ranked_df['win'].isnull()]['matchid']
lol_ranked_df = lol_ranked_df[~lol_ranked_df['matchid'].isin(missing_records)]

In [42]:
## No missing records now
lol_ranked_df.isnull().sum()

id             0
matchid        0
player         0
name           0
platformid     0
              ..
pinksbought    0
wardsbought    0
wardsplaced    0
wardskilled    0
firstblood     0
Length: 70, dtype: int64

In [44]:
print(f"Final data has {lol_ranked_df['matchid'].nunique()} games")

Final data has 148635 games


In [45]:
## Save Data
lol_ranked_df.to_csv("C:\\Users\\Admin\\Desktop\\upwork_jobs\\lol_project\\data\\lol_ranked_data_merged.csv",index=False)